# Aim

Get FFAST working at a basic level (no noise, no window_var, etc.).

- for reasonably high-snr like around 5db (without the bin-length gain) look at kay vs. kay2 my guess is kay will fail but kay2 will keep working.  how much db can you go lower with kay2. is kay2 alway better?
- how does all this compare to "new" method?
- keep an eye on the total number of samples used by each method.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")
from ffast import *
from matplotlib import pyplot as plt
import numpy as np
import time

from tqdm import tqdm_notebook as tqdm

- making the seed of delays 0 makes the non-singleton locations pass the singleton test wrongly

In [2]:
7**3

343

In [3]:
19**2

361

In [4]:
params = make_args()

params.primes = [7, 19]
params.prime_powers = [3, 2]

params.bins = [i**j for i,j in zip(params.primes, params.prime_powers)]
print(params.bins)

params.length = np.prod(params.bins)
print(np.log2(params.length))

params.iterations = 50
params.snr = 3 - 10*np.log10(params.bins[1])
# params.snr = -22

print('snr: {}'.format(params.snr))

params.sparsity = 1

# for ml
# params.bin_processing_method = 'ml'
# params.chains = 500
# params.delays = 1

# for kay
# params.bin_processing_method = 'kay'
params.bin_processing_method = 'kay2'
params.chains = 16 # needs to be int(np.floor(np.log2(params.length))) [rounds down]
params.delays = 3 # parameter to play with

# for new
# params.bin_processing_method = 'new'
# params.chains = 1 # this is set to 1 always
# params.delays = 1 # parameter to play with [the larger this value the larger the gap should be]



config = Config(params)
config.compute_params()

input_signal = ExperimentInputSignal(config)
input_signal.process()
output_signal = ExperimentOutputSignal(config, input_signal)

[343, 361]
16.917919793059983
snr: -22.575072019056577


In [5]:
np.sum(np.array(params.primes)*np.array(params.prime_powers))*params.delays

177

In [6]:
params.chains*params.delays

48

In [7]:
config.get_noise_sd()**2

180.9285913394453

In [8]:
# vars(config)

In [9]:
config.display()

Running experiment mode (for now)
Signal length: 123823
Signal sparsity: 1
Signal-to-noise ratio (dB): -22
Random phase
Delays: 48
Bins: 343 361
Bin processor: kay2


In [10]:
# plt.plot(np.real(input_signal.time_signal))
# plt.plot(np.imag(input_signal.time_signal))
# plt.show()

In [11]:
# len(input_signal.time_signal)

In [12]:
# signal_fft = np.fft.fft(input_signal.time_signal)
# plt.plot(np.abs(signal_fft))
# plt.title('loc: {}'.format(input_signal.freqs))
# plt.show()

In [13]:
# input_signal.time_signal *= 2 # demonstrating that the input signal can be changed

In [14]:
ffast = FFAST(config, input_signal, output_signal)

In [15]:
ffast.frontend.process()
print(ffast.frontend.get_used_samples_nb()/ config.signal_length)

0.24190174684832383


In [16]:
params.bins

[343, 361]

In [17]:
ffast = FFAST(config, input_signal, output_signal)

if not config.help_displayed:
    # this disregards the time to make the signal
    time_initial = int(np.round(time.time() * 1000)) 

    iterations = config.iterations
    
    for i in tqdm(range(iterations)):
        input_signal.process() # ffast.get_delays() in the CPP code as an argument
        print('input freq: {}'.format(input_signal.freqs))
        
        print('true singleton bins:')
        for stage_idx, bin_size in enumerate(params.bins):
            print('    stage {} bin {}'.format(stage_idx, input_signal.freqs[0] % bin_size))
        
        ffast.process()
        output_signal.process()
        print('---')

    time_final = int(np.round(time.time() * 1000))

    ffast.display_results(time_final - time_initial)

input freq: [98343]
true singleton bins:
    stage 0 bin 245
    stage 1 bin 151
found a singleton at 98343 -- stage 1 -- bin 151
---
input freq: [65400]
true singleton bins:
    stage 0 bin 230
    stage 1 bin 59
found a singleton at 65400 -- stage 0 -- bin 230
---
input freq: [61576]
true singleton bins:
    stage 0 bin 179
    stage 1 bin 206
found a singleton at 61576 -- stage 0 -- bin 179
---
input freq: [21648]
true singleton bins:
    stage 0 bin 39
    stage 1 bin 349
found a singleton at 21648 -- stage 0 -- bin 39
---
input freq: [119679]
true singleton bins:
    stage 0 bin 315
    stage 1 bin 188
found a singleton at 119679 -- stage 0 -- bin 315
---
input freq: [63391]
true singleton bins:
    stage 0 bin 279
    stage 1 bin 216
---
input freq: [85121]
true singleton bins:
    stage 0 bin 57
    stage 1 bin 286
found a singleton at 85121 -- stage 0 -- bin 57
---
input freq: [10796]
true singleton bins:
    stage 0 bin 163
    stage 1 bin 327
found a singleton at 10796 -- sta

In [18]:
output_signal.statistics()[0]/params.iterations

0.88

# this part is for debugging

In [20]:
vars(ffast.config)

{'output_file': None,
 'signal_length': 123823,
 'signal_length_original': 123823,
 'signal_sparsity_peeling': 1,
 'signal_sparsity': 1,
 'length_factor': 1,
 'maximum_likelihood': False,
 'count_samples': True,
 'delays_per_bunch_nb': 3,
 'chains_nb': 16,
 'off_grid_SNR_dB': 100,
 'min_fourier_magnitude': 1,
 'help_displayed': False,
 'primes': array([ 7, 19]),
 'prime_powers': array([3, 2]),
 'bins': array([343, 361]),
 'distribution': array([0., 1.]),
 'iterations': 50,
 'experiment_mode': False,
 'phases_nb': 0,
 'display_iteration_time': False,
 'noisy': True,
 'quantize': False,
 'SNR_dB': -22.575072019056577,
 'quantization_bits_nb': 0,
 'max_SNR_dB': -22.575072019056577,
 'verbose': False,
 'reconstruct_signal_in_backend': False,
 'off_grid': False,
 'default_delays': False,
 'apply_window_var': False,
 'input_file': '/Users/adityasengupta/Library/Jupyter/runtime/kernel-ca65e5ed-bf40-4a01-872a-b8544b5bd12d.json',
 'bin_processing_method': 'kay2',
 'bins_sum': 704,
 'bin_offsets

In [21]:
def test_many(f, num_iter, **kwargs):
    r = []
    for i in tqdm(range(num_iter)):
        r.append(f(**kwargs))
    return r

In [22]:
def test_if_location_is_true(stage_idx):
    input_signal.process()
    ffast.frontend.process()
    singleton_bin = input_signal.freqs[0] % params.bins[stage_idx]
    binprocessor = BinProcessor(config, ffast.frontend.delays, ffast.frontend.observation_matrix)
    
    binprocessor.adjust_to(config.bin_offsets[stage_idx]+singleton_bin, singleton_bin, stage_idx)
    binprocessor.process()
    loc_hat = binprocessor.location
    loc = input_signal.freqs[0]
    if loc_hat == loc:
        return True
    else:
        return False

In [23]:
results_of_location_estimation = test_many(test_if_location_is_true, 100, stage_idx=0)
print('location identification accuracy: {}'.format(np.mean(results_of_location_estimation)))


location identification accuracy: 0.59


In [24]:
input_signal.process() # ffast.get_delays() in the CPP code as an argument
print("true frequencies: {}".format(input_signal.freqs))

ffast.frontend.process()
# ffast.backend.process()
# output_signal.process()

true frequencies: [102329]


In [ ]:
# plt.stem(np.linalg.norm(ffast.frontend.observation_matrix, axis=0)**2/ffast.config.delays_nb)
# plt.xlabel('bin absolute index')
# plt.show()

In [25]:
singleton_bin = input_signal.freqs[0] % params.bins[0]
print(singleton_bin)

115


In [26]:
binprocessor = BinProcessor(config, ffast.frontend.delays, ffast.frontend.observation_matrix)

In [27]:
# plt.stem(np.linalg.norm(ffast.frontend.observation_matrix, axis=0)**2/ffast.config.delays_nb)
# plt.axhline(binprocessor.thresholds[0])
# plt.xlabel('bin absolute index')
# plt.show()

In [28]:
binprocessor.adjust_to(singleton_bin, singleton_bin, 0)

In [29]:
binprocessor.is_zeroton()

energy: 60.08823593068651
threshold: 361.85718267899057


True

In [30]:
binprocessor.process()

In [31]:
# binprocessor.find_location_new()
print(binprocessor.location)
print(input_signal.freqs)

102329
[102329]


In [32]:
binprocessor.is_singleton()

True

In [33]:
binprocessor.location

102329

In [34]:
binprocessor.amplitude

(0.879865872482332-0.05884119326634166j)

In [35]:
binprocessor.noise

0.4742120089718666

In [36]:
binprocessor.thresholds[0]

361.85718267899057

In [37]:
binprocessor.minimum_energy

0.1

In [38]:
ffast.backend.decoded_frequencies

{27728: (1.0189003610731628+0.0699427304035451j)}

In [39]:
output_signal.check_full_recovery()

In [40]:
output_signal.statistics()

(44, 0)

In [41]:
ffast = FFAST(config, input_signal, output_signal)

In [42]:
input_signal.process() # ffast.get_delays() in the CPP code as an argument
print(input_signal.freqs)

[102364]


In [43]:
ffast.process()

found a singleton at 102364 -- stage 0 -- bin 150


In [44]:
output_signal.process()

In [45]:
output_signal.statistics()

(45, 0)